In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

train_data = pd.read_csv('C:/Users/atalb/Documents/Coding/MLandAI/MLPracticeExercises/Datasets/Titanic/train.csv')
test_data = pd.read_csv('C:/Users/atalb/Documents/Coding/MLandAI/MLPracticeExercises/Datasets/Titanic/test.csv')

train_data.head()



,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [38]:
# Combine the two dataset (this helps to make sure that they will stay consistent after implementing feature engineering)
# Add a column to indicate the dataset type
train_data["DatasetType"] = "train"
test_data["DatasetType"] = "test"

# Combine train and test
combined_data = pd.concat([train_data, test_data], axis=0).reset_index(drop=True)

In [39]:
def data_overview(df, head_rows=5):
    """
    Provides a comprehensive overview of a DataFrame, including:
    - Shape of the DataFrame
    - First few rows (default 5)
    - Info (data types, non-null counts)
    - Basic descriptive statistics
    - Missing values (count and percentage)
    - Duplicate rows count
    - Number of unique values and the unique values for each column
    """
    print("======== DATA OVERVIEW ========")
    print(f"\n1. Shape of DataFrame: {df.shape[0]} rows, {df.shape[1]} columns\n")

    print(f"2. First {head_rows} Rows:\n")
    display(df.head(head_rows))

    print("\n3. DataFrame Info:\n")
    print(df.info())

    print("\n4. Descriptive Statistics (Numerical Columns):\n")
    print(df.describe().transpose())

    print("\n5. Missing Values Count and Percentage:\n")
    missing_values = df.isnull().sum()
    missing_percentage = (df.isnull().sum() / len(df)) * 100
    missing_data = pd.DataFrame({
        'Missing Count': missing_values,
        'Missing Percentage (%)': missing_percentage
    })
    missing_data = missing_data[missing_data["Missing Count"] > 0].sort_values(by='Missing Percentage (%)', ascending=False)
    
    if not missing_data.empty:
        display(missing_data)
    else:
        print("No missing values")

    print("\n6. Duplicate Rows Count:\n")
    duplicate_count = df.duplicated().sum()
    print(f"{duplicate_count} duplicate rows found\n" if duplicate_count > 0 else "No duplicate rows found\n")

    print("\n7. Number of Unique Values per Column:\n")
    unique_counts = df.nunique()
    unique_values = {col: df[col].unique() for col in df.columns}
    for col in unique_counts.index:
        print(f"{col}: {unique_counts[col]} unique values")
        if unique_counts[col] <= 10:  # Print unique values only if there are <= 10 unique values for readability
            print(f"   Unique Values: {unique_values[col]}\n")
        else:
            print("   Too many unique values to display\n")

  


In [41]:
data_overview(combined_data)


======== DATA OVERVIEW ========

1. Shape of DataFrame: 1309 rows, 13 columns

2. First 5 Rows:



,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,DatasetType
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,train
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,train
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,train
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,train
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,train



3. DataFrame Info:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Survived     891 non-null    float64
 2   Pclass       1309 non-null   int64  
 3   Name         1309 non-null   object 
 4   Sex          1309 non-null   object 
 5   Age          1046 non-null   float64
 6   SibSp        1309 non-null   int64  
 7   Parch        1309 non-null   int64  
 8   Ticket       1309 non-null   object 
 9   Fare         1308 non-null   float64
 10  Cabin        295 non-null    object 
 11  Embarked     1307 non-null   object 
 12  DatasetType  1309 non-null   object 
dtypes: float64(3), int64(4), object(6)
memory usage: 133.1+ KB
None

4. Descriptive Statistics (Numerical Columns):

              count        mean         std   min       25%       50%  \
PassengerId  1309.0  655.000000  378.020061  1.00  

,Missing Count,Missing Percentage (%)
Cabin,1014,77.463713
Survived,418,31.932773
Age,263,20.091673
Embarked,2,0.152788
Fare,1,0.076394



6. Duplicate Rows Count:

No duplicate rows found


7. Number of Unique Values per Column:

PassengerId: 1309 unique values
   Too many unique values to display

Survived: 2 unique values
   Unique Values: [ 0.  1. nan]

Pclass: 3 unique values
   Unique Values: [3 1 2]

Name: 1307 unique values
   Too many unique values to display

Sex: 2 unique values
   Unique Values: ['male' 'female']

Age: 98 unique values
   Too many unique values to display

SibSp: 7 unique values
   Unique Values: [1 0 3 4 2 5 8]

Parch: 8 unique values
   Unique Values: [0 1 2 5 3 4 6 9]

Ticket: 929 unique values
   Too many unique values to display

Fare: 281 unique values
   Too many unique values to display

Cabin: 186 unique values
   Too many unique values to display

Embarked: 3 unique values
   Unique Values: ['S' 'C' 'Q' nan]

DatasetType: 2 unique values
   Unique Values: ['train' 'test']



In [43]:
# Converting data types
combined_data['Survived'] = combined_data['Survived'].astype('category')
combined_data['Pclass'] = combined_data['Pclass'].astype('category')
combined_data['Sex'] = combined_data['Sex'].astype('category')
combined_data['Cabin'] = combined_data['Cabin'].astype('category')
combined_data['Embarked'] = combined_data['Embarked'].astype('category')
combined_data['Name'] = combined_data['Name'].astype('category')
combined_data['Ticket'] = combined_data['Ticket'].astype('category')
combined_data['DatasetType'] = combined_data['DatasetType'].astype('category')
combined_data['PassengerId'] = combined_data['PassengerId'].astype('category')

combined_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   PassengerId  1309 non-null   category
 1   Survived     891 non-null    category
 2   Pclass       1309 non-null   category
 3   Name         1309 non-null   category
 4   Sex          1309 non-null   category
 5   Age          1046 non-null   float64 
 6   SibSp        1309 non-null   int64   
 7   Parch        1309 non-null   int64   
 8   Ticket       1309 non-null   category
 9   Fare         1308 non-null   float64 
 10  Cabin        295 non-null    category
 11  Embarked     1307 non-null   category
 12  DatasetType  1309 non-null   category
dtypes: category(9), float64(2), int64(2)
memory usage: 188.4 KB


In [44]:
import re
# Function to extract title from the Name
def extract_title(Name):
    title_search = re.search(r',\s*([^\.]*)\.', Name)
    if title_search:
        return title_search.group(1).strip()
    return ""

# Function to extract family name from the Name
def extract_family_name(Name):
    family_name = Name.split(",")[0].strip()
    return family_name

# Create new columns for Title and Family Name
combined_data["Title"] = combined_data["Name"].apply(extract_title).astype('category')
combined_data["FamilyName"] = combined_data["Name"].apply(extract_family_name).astype('category')

# check the new columns
print(combined_data["Title"].unique())
print(combined_data["FamilyName"].unique())

['Mr', 'Mrs', 'Miss', 'Master', 'Don', ..., 'Col', 'Capt', 'the Countess', 'Jonkheer', 'Dona']
Length: 18
Categories (18, object): ['Capt', 'Col', 'Don', 'Dona', ..., 'Ms', 'Rev', 'Sir', 'the Countess']
['Braund', 'Cumings', 'Heikkinen', 'Futrelle', 'Allen', ..., 'Naughton', 'Henriksson', 'Spector', 'Oliva y Ocana', 'Saether']
Length: 875
Categories (875, object): ['Abbing', 'Abbott', 'Abelseth', 'Abelson', ..., 'de Pelsmaeker', 'del Carlo', 'van Billiard', 'van Melkebeke']


In [45]:
# Grouping titles into broader categories
def encode_title(title):
    if title in ['Mr', 'Mrs', 'Miss', 'Master']:
        return title  # Keep common titles as they are
    elif title in ['Mlle', 'Ms']:  # Alternative spellings for Miss
        return 'Miss'
    elif title in ['Mme']:  # Alternative spelling for Mrs
        return 'Mrs'
    elif title in ['Dr', 'Major', 'Col', 'Capt', 'Rev']:
        return 'Officer'  # Group into an 'Officer' category
    elif title in ['Don', 'Sir', 'the Countess', 'Lady', 'Jonkheer']:
        return 'Noble'  # Group into a 'Noble' category
    else:
        return 'Other'  # Catch-all for any unusual titles

# Apply title encoding
combined_data["TitleEncoded"] = combined_data["Title"].apply(encode_title).astype('category')

# Check the new column
combined_data["TitleEncoded"].unique()

['Mr', 'Mrs', 'Miss', 'Master', 'Noble', 'Officer', 'Other']
Categories (7, object): ['Master', 'Miss', 'Mr', 'Mrs', 'Noble', 'Officer', 'Other']

In [ ]:
# Create a "FamilySize" feature by summing SibSp (siblings/spouses) and Parch (parents/children) plus 1 (the individual themselves)
combined_data["FamilySize"] = combined_data["SibSp"] + combined_data["Parch"] + 1

# Create a "FamilyGroup" feature based on FamilySize
def categorize_family_size(size):
    if size == 1:
        return "Solo"  # No family members on board
    elif size <= 4:
        return "Small"  # Small family (1-3 additional members)
    else:
        return "Large"  # Large family (4+ additional members)

combined_data["FamilyGroup"] = combined_data["FamilySize"].apply(categorize_family_size).astype('category')

# Check the new column
combined_data["FamilyGroup"].unique()